In [12]:
import tdmclient.notebook
import numpy as np
await tdmclient.notebook.start()

In [13]:
%%run_python
timer_period[0] = 10  # 10ms sampling time
timer=0
@onevent 
def timer0():
    global prox_horizontal, motor_left_target, motor_right_target, timer
    SPEED=100
    PROX_FACTOR=80
    THRESHOLD_PROX=50
    if prox_horizontal[0]>THRESHOLD_PROX or prox_horizontal[1]>THRESHOLD_PROX:
        #Should turn right
        motor_left_target=(SPEED+prox_horizontal[0]//PROX_FACTOR+prox_horizontal[1]//PROX_FACTOR)
        motor_right_target=(SPEED-prox_horizontal[0]//PROX_FACTOR-prox_horizontal[1]//PROX_FACTOR)
        timer=300
    elif prox_horizontal[4]>THRESHOLD_PROX :#or prox_horizontal[3]>THRESHOLD_PROX:
        #####################################################Code claqué sinon fonctionne pas
        #if prox_horizontal[4]>THRESHOLD_PROX:
        motor_left_target=(SPEED-prox_horizontal[4]//PROX_FACTOR-prox_horizontal[3]//PROX_FACTOR)
        motor_right_target=(SPEED+prox_horizontal[4]//PROX_FACTOR+prox_horizontal[3]//PROX_FACTOR)
        timer=300
        #if prox_horizontal[3]>THRESHOLD_PROX:
    elif prox_horizontal[3]>THRESHOLD_PROX:
        #Should turn left
        motor_left_target=(SPEED-prox_horizontal[4]//PROX_FACTOR-prox_horizontal[3]//PROX_FACTOR)
        motor_right_target=(SPEED+prox_horizontal[4]//PROX_FACTOR+prox_horizontal[3]//PROX_FACTOR)
        timer=300
    else:
        if timer>0:
            timer=timer-1
            motor_left_target=SPEED
            motor_right_target=SPEED
        else:
            timer=0
    
@onevent
def button_center():
    global motor_left_target, motor_right_target
    motor_left_target=0
    motor_right_target=0

In [14]:
await tdmclient.notebook.stop()

In [28]:
from tdmclient import ClientAsync
client = ClientAsync()
node = await client.wait_for_node()

In [32]:
def motors_command(motors):
    return {
        "motor.left.target": [motors[0]],
        "motor.right.target": [motors[1]],
    }

await node.lock()
node.send_set_variables(motors_command((0,0)))
await node.unlock()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte

In [6]:
import numpy as np
from numpy.lib.function_base import angle

from tdmclient import ClientAsync
client = ClientAsync()


THRESHOLD_POS=1     #Has to be tuned
THRESHOLD_ANGLE=10
SPEED=30
P=10
D=0.001


#Navigation

#Output Kalman: Position, angle, vitesse, vitesse angulaire

#Output path planning: Prochaine position

def motors_command(motors):
    return {
        "motor.left.target": [motors[0]],
        "motor.right.target": [motors[1]],
    }



def angle(P1,P2):
    angle=np.arctan2(P2[1]-P1[1],P2[0]-P1[0])
    #right half
    if P2[0]>=P1[0]:
        #Up
        if P2[1]>=P1[1]:
            return angle
        #Bot
        else:
            return angle+2*np.pi
    #left half
    else:
        #Up
        if P2[1]>=P1[1]:
            return angle+np.pi
        else:
            return angle+(3/2)*np.pi


def path_side(dep, obj, pos_r):
    v_path=np.array([obj[0]-dep[0],obj[1]-dep[1]])
    v_robot=np.array([pos_r[0]-dep[0],pos_r[1]-dep[1]])
    vect_prod=np.cross(v_path,v_robot)
    #left
    if vect_prod>=0:
        return True
    #right
    else:
        return False
#return the position error
def error(pos_r, dep, obj):
    error=np.linalg.norm(np.cross(obj-dep, dep-pos_r))/np.linalg.norm(obj-dep)
    return error

#When the objectif is reached, one must receive a new objectif and the precedent objectif becomes the depart position
def reach_obj(pos_r, obj, objectif_number, obj_list):    
    if pos_r[0]>=obj[0]-THRESHOLD_POS and pos_r[0]<=obj[0]+THRESHOLD_POS:
        if pos_r[1]>=obj[1]-THRESHOLD_POS and pos_r[1]<=obj[1]+THRESHOLD_POS:
            next_obj(obj_list, objectif_number)
            return True
    else:
        return False

#When on an objectif, choose the optimal side to turn
def optimal_side(angle_r, dep, obj):
    rad=angle(dep,obj)

    #right
    if abs(angle_r-rad) <= abs(2*np.pi-angle_r+rad):
        return False
    else:
        return True

#When an objectif is reached, the robot turn on itself until it reaches the good angle.
def start_angle(angle_r, dep, obj, pos_r):
    rad = angle(dep, obj)
    #The angle is good
    if angle_r>=rad-THRESHOLD_ANGLE and angle_r<=rad+THRESHOLD_ANGLE:
        return np.array([SPEED,SPEED])
        #left
    else:
        if optimal_side(angle_r, dep, obj):
            motors=np.array([SPEED,-SPEED])
            #err_pos=0
        else:
            motors=np.array([-SPEED,SPEED])
            #err_pos=0
        
        return motors
        #err_pos


#Modify dep and obj
def next_obj(obj_list, i):
    dep=obj_list[i]
    obj=obj_list[i+1]
    return dep, obj



#Je pense qu'un regulateur P devrait suffire....
def motors_corr(prev_err_pos, err_pos, dep, obj, pos_r, T):

    speed_corr=err_pos*P  + D*(err_pos-prev_err_pos)/T
    motors=np.array([0,0])

    if path_side(dep, obj, pos_r):
        #left
        
        motors[0]=SPEED+speed_corr
        motors[1]=SPEED-speed_corr
    else:
        motors[0]=SPEED-speed_corr
        motors[1]=SPEED+speed_corr
    

    return motors



def end(objectif_number, obj_list, pos_r):
    if objectif_number==len(obj_list) and reach_obj(pos_r, obj_list[objectif_number-1], objectif_number, obj_list):
        return np.array([0,0])
    else:
        return np.array([SPEED,SPEED])
    
def navigation(pos_r, angle_r, obj_list, prev_err_pos, T, objectif_number, prox_horizontal):
    dep, obj=next_obj(obj_list, objectif_number)

    err_pos=error(pos_r, dep, obj)
    motors=motors_corr(prev_err_pos, err_pos, dep, obj, pos_r, T)
    
    rad = angle(dep, obj)
    if reach_obj(pos_r, obj, objectif_number, obj_list):
        motors=start_angle(angle_r, dep, obj, pos_r)


    #return motors, err_pos, objectif_number
    #send_command_motors(motors)
    motors=end(objectif_number, obj_list, pos_r);
    return motors

In [86]:
node = await client.wait_for_node()
while 1:

    await node.lock()
    node.send_set_variables(motors_command(navigation(np.array([0,0]), 0, np.array([[0, 0], [10,0]]), 0, 0.01, 0, prox_horizontal)))
    await node.unlock()



Exception: Node lock error

In [87]:
await node.lock()
node.send_set_variables((0,0))
await node.unlock()

Exception: Node lock error

Exception in thread Thread-39:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Exception in thread Exception in thread Thread-47:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-42:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-45:
Traceback (most recent call last):
Exception in thread Thread-41:
Thread-44  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Exception in thread Thread-43:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line 973, in _bootstrap_inner
Thread-40:
Traceback (most recent call last):
  File "C:\Users\Valentin Roch\Desktop\python\lib\threading.py", line

    return self.socket.recv(n)
ConnectionResetError: [WinError 10054] Une connexion existante a dû être fermée par l’hôte distant


In [39]:
import tdmclient.notebook
#import numpy as np
await tdmclient.notebook.start()

In [2]:
%%run_python
#Manual Calibration

PERIOD=10
MS_TO_S=1000

#Rotation
count=0
time=0

timer_period[0]=PERIOD
#timer_period[1]=9000


@onevent
def button_right():
    global motor_left_target, motor_right_target, count, time, timer_period
    count=1
    motor_left_target=100
    motor_right_target=-100
    timer_period[1]=9000
    
@onevent
def button_forward():
    global motor_left_target, motor_right_target, count, time, timer_period
    motor_left_target=100
    motor_right_target=100
    timer_period[1]=9000
    time=0
    count=0
#30cm en 9s
    
@onevent
def button_center():
    global count,time, PERIOD, MS_TO_S, motor_left_target, motor_right_target
    print(time*PERIOD)
    count=0
    time=0
    motor_left_target=0
    motor_right_target=0
    
@onevent
def button_backward():
    global count,time, PERIOD, MS_TO_S, motor_left_target, motor_right_target
    print(time)
    
@onevent
def timer0():
    global count, time, motor_left_target, motor_right_target
    if count==1:
        time=time+1
        if time*PERIOD>9133:
            motor_left_target=0
            motor_right_target=0

#@onevent
#def timer1():
#    global motor_left_target, motor_right_target
#    motor_left_target=0
#    motor_right_target=0
#
    
#9133 ms for one turn

In [21]:
async def print_sensor_values(s):   
    await node.lock()
    await node.wait_for_variables({str(s)})
    return node[s]
    await node.unlock()
motor=await print_sensor_values('motor.target')
await node.lock()
mot= await node.wait_for_variables({'motor.left.target'})
await node.unlock()

print(mot)
print(motor)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 1: invalid continuation byte

In [100]:
await tdmclient.notebook.stop()

ValueError: Function <function _pre_run_cell at 0x000001D22CD9EEE0> is not registered as a pre_run_cell callback

In [61]:
import thymio
thymio.leds_top = thymio.RED

ModuleNotFoundError: No module named 'thymio'